In [1]:
import pandas as pd
import requests
import os
import time

# Extracción de Datos de Clima

In [ ]:
# --- CONFIGURACIÓN ---
# Cambiar rutas según sea necesario
ruta_excel = r"C:\Users\glova\OneDrive\Documentos\clima\Ubicaciones.xlsx"
carpeta_resultados = r"C:\Users\glova\OneDrive\Documentos\Clima\resultados_clima"
carpeta_errores = r"C:\Users\glova\OneDrive\Documentos\Clima\errores"
anios = [2019, 2024]

# Crear carpetas si no existen
os.makedirs(carpeta_resultados, exist_ok=True)
os.makedirs(carpeta_errores, exist_ok=True)

# Leer ciudades
df = pd.read_excel(ruta_excel)
errores = []

# --- PROCESO PRINCIPAL ---
for _, fila in df.iterrows():
    ciudad_id = fila["id"]
    lat = fila["lat"]
    lon = fila["lng"]
    ciudad = fila["city"]
    pais = fila["country"]

    for anio in anios:
        nombre_archivo = f"{ciudad_id}_{anio}_clima.csv"
        ruta_archivo = os.path.join(carpeta_resultados, nombre_archivo)

        if os.path.exists(ruta_archivo):
            continue

        print(f"📥 Descargando clima para {ciudad_id} - {ciudad} ({anio})")

        url = (
            f"https://archive-api.open-meteo.com/v1/archive?"
            f"latitude={lat}&longitude={lon}"
            f"&start_date={anio}-01-01&end_date={anio}-12-31"
            f"&daily=temperature_2m_mean,relative_humidity_2m_mean"
            f"&timezone=auto"
        )

        try:
            r = requests.get(url)
            data = r.json()

            # Detectar error por límite diario
            if data.get("error") and "limit" in data.get("reason", "").lower():
                print("🛑 Límite diario alcanzado. Deteniendo ejecución.")
                raise SystemExit("⛔ Límite diario de la API alcanzado")

            if "daily" in data and data["daily"].get("temperature_2m_mean"):
                df_diario = pd.DataFrame(data["daily"])
                df_diario.to_csv(ruta_archivo, index=False, encoding="utf-8-sig")
                print(f"✅ Guardado: {ruta_archivo}")
            else:
                raise ValueError("❌ No se encontraron datos climáticos válidos")

        except Exception as e:
            errores.append({
                "id": ciudad_id,
                "ciudad": ciudad,
                "pais": pais,
                "anio": anio,
                "error": str(e)
            })
            print(f"⚠ Error para {ciudad_id} - {ciudad}: {e}")

        time.sleep(0.3)  # Evitar saturar la API

# --- GUARDAR ERRORES ---
if errores:
    errores_path = os.path.join(carpeta_errores, "errores_clima.xlsx")
    pd.DataFrame(errores).to_excel(errores_path, index=False)
    print(f"⚠ Errores guardados en: {errores_path}")

print("✅ Proceso finalizado o detenido por límite diario.")


# Extracción de Datos de Calidad del Aire

Se extrajo datos del 2019 y 2024. Sin embargo, el histórico para los datos de calidad del aire de 2019 no se encontraba disponible, así se haya comprado una licencia (API KEY).

In [ ]:
# --- CONFIGURACIÓN ---
API_KEY = "LI3LHikgqN3Wf1qd"
ruta_excel = r"C:\Users\glova\OneDrive\Documentos\clima\Ubicaciones.xlsx"
carpeta_resultados = r"C:\Users\glova\OneDrive\Documentos\nuevo_ciudades\resultados"
carpeta_errores = r"C:\Users\glova\OneDrive\Documentos\nuevo_ciudades\errores"
anios = [2019, 2024]

# Crear carpetas si no existen
os.makedirs(carpeta_resultados, exist_ok=True)
os.makedirs(carpeta_errores, exist_ok=True)

# Leer ciudades
df = pd.read_excel(ruta_excel)
errores = []

# --- PROCESO ---
for _, fila in df.iterrows():
    ciudad_id = fila["id"]
    lat = fila["lat"]
    lon = fila["lng"]
    ciudad = fila["city"]
    pais = fila["country"]

    for anio in anios:
        nombre_archivo = f"{ciudad_id}_{anio}_aire.csv"
        ruta_archivo = os.path.join(carpeta_resultados, nombre_archivo)

        # Saltar si ya fue procesado
        if os.path.exists(ruta_archivo) and os.path.getsize(ruta_archivo) > 500:
            continue

        print(f"📥 Descargando calidad del aire para {ciudad_id} ({ciudad}) - {anio}")

        url = (
            f"https://customer-api.open-meteo.com/v1/air-quality?"
            f"latitude={lat}&longitude={lon}"
            f"&start_date={anio}-01-01&end_date={anio}-12-31"
            f"&daily=pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,dust"
            f"&timezone=auto"
            f"&apikey={API_KEY}"
        )

        try:
            r = requests.get(url)
            data = r.json()

            # Detectar error por límite
            if data.get("error") and "limit exceeded" in data.get("reason", "").lower():
                print("🛑 Límite diario alcanzado. Deteniendo el proceso.")
                raise SystemExit

            if "daily" in data and data["daily"].get("pm10"):
                pd.DataFrame(data["daily"]).to_csv(ruta_archivo, index=False, encoding="utf-8-sig")
                print(f"✅ Guardado: {ruta_archivo}")
            else:
                raise ValueError("Datos vacíos o malformateados")

        except Exception as e:
            errores.append({
                "id": ciudad_id,
                "ciudad": ciudad,
                "pais": pais,
                "anio": anio,
                "error": str(e)
            })

        time.sleep(0.3)  # moderar frecuencia de llamadas

# --- GUARDAR ERRORES ---
if errores:
    errores_path = os.path.join(carpeta_errores, "errores_calidad_aire.xlsx")
    pd.DataFrame(errores).to_excel(errores_path, index=False)
    print(f"⚠ Errores registrados en: {errores_path}")

print("✅ Proceso completado (o detenido por límite diario).")